In [1]:
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
###Helper functions
def __parse_codebook_data(fname):
    fNames = dict()
    with open(fname, 'r') as fl:
        lstate = False
        for line in fl:
            if line.startswith('-----'):
                lstate = True
            else:
                if lstate:
                    while '  ' in line:
                        line = line.replace('  ', ' ')
                    #print line
                    temp = line.strip().split(' ')
                    fNames[temp[0]] = ' '.join(temp[1:])
                lstate = False
    return fNames

def feature_desc_contains(term,output=True):
    features_list = []
    featureNames = dict()
    # Looking for a path that has all feature description files
    for fname in glob.glob('../data/raw_FFChallenge_v2/codebook_FFChallenge.txt'):
        featureNames.update(__parse_codebook_data(fname))
    for f in featureNames:
        if term in featureNames[f]:
            if output:
                print('[{}]: {}'.format(f, featureNames[f]))
            features_list.append(f)
    return features_list

def desc_feature(term,output=True):
    featureNames = dict()
    # Looking for a path that has all feature description files
    for fname in glob.glob('../data/raw_FFChallenge_v2/codebook_FFChallenge.txt'):
        featureNames.update(__parse_codebook_data(fname))
    for f in featureNames:
        if term == f:
            if output:
                print('[{}]: {}'.format(f, featureNames[f]))
            return featureNames[f]

In [3]:
df = pd.read_csv('../data/raw_FFChallenge_v2/background.csv',index_col="challengeID")
variables = df.columns
df[df < 0] = np.nan
n_col = len(df.columns)
n_rows = len(df)
n_col

12942

In [4]:
##generally, remove all data has no variance or have more than 80 percent missing data
count = 0
zero_variance_col = []
badly_missing = []
exceptions = []
potential_identifier = []
for x in variables:
    try:
        count+=1
        if count == 1000:
            print(count)
            count=0
        if x == 'challengeID':
            continue    
        ##remove if there is no standard deviation or missing more than 80% of the data
        unique_values = len(df[x].unique())
        if unique_values ==1: #no variance -- extereme case that captures also strings
            df.drop(x, axis=1, inplace=True)
            zero_variance_col.append(x)
        elif unique_values > n_rows-200: #potential identifiers
            df.drop(x, axis=1, inplace=True)
            potential_identifier.append(x)
        elif sum(pd.isnull(df[x])) > n_rows*0.8:
            df.drop(x, axis=1, inplace=True)
            badly_missing.append(x)
        elif df[x].std(skipna=True) < 0.05:
            df.drop(x, axis=1, inplace=True)
            zero_variance_col.append(x)
    except Exception as ex:
        exceptions.append(x)
        print('in feature:',x)
        print(ex)
        print('###')
len(df.columns)

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


5167

In [5]:
len(df.columns)

5167

In [6]:
df.to_csv('./data_cleaned_stage1_withNans.csv',index_label='challengeID')

In [7]:
potential_identifier

[]

In [18]:
subfields = list(df.columns)
subfields.append('challengeID')
subfieldas = list(set(subfields))
df1 = pd.read_csv('../data/raw_FFChallenge_v2/background.csv',index_col="challengeID",usecols=subfields)
df1.to_csv('./data_cleaned_stage1_withMissingCodes.csv',index_label='challengeID')

In [ ]:
d = pd.read_csv('../data/raw_FFChallenge_v2/background.csv')
d

In [ ]:
'challengeID' in df.columns

In [16]:
subfields = list(df.columns)
subfields.append('challengeID')
subfieldas = list(set(subfields))